<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [4]:
df = df.sample(False, 0.5)

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [6]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [7]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [9]:
index_columns = [i+'_index' for i in cat_columns]
string_indexers = [StringIndexer(inputCol=cat, outputCol=ind) for cat, ind in zip(cat_columns, index_columns)]

In [10]:
ohe_columns = [i+'_ohe' for i in cat_columns]
ohe_estimator = OneHotEncoderEstimator(inputCols=index_columns, outputCols=ohe_columns)

In [11]:
vector_assembler = VectorAssembler(inputCols=num_columns+ohe_columns, outputCol='features')

In [12]:
pipeline = Pipeline(stages=[*string_indexers,
                                            ohe_estimator,
                                            vector_assembler])

### Fit and save pipeline

In [13]:
model = pipeline.fit(df)

In [14]:
model.write().overwrite().save(DATA_PATH+'/pipeline_model')

### Load fitted pipeline

In [15]:
from pyspark.ml import PipelineModel


In [16]:
loaded_pipeline_model = PipelineModel.load(DATA_PATH+'/pipeline_model')

### Transform dataset using pipeline

In [17]:
df = loaded_pipeline_model.transform(df)

In [18]:
df = df.select('id', 'features', F.col('_c0').alias('label'))

In [19]:
df.take(5)

[Row(id=26, features=SparseVector(1952, {1: 1.0, 2: 20.0, 3: 16.0, 4: 1548.0, 5: 93.0, 6: 42.0, 7: 32.0, 8: 912.0, 10: 15.0, 11: 1.0, 12: 16.0, 16: 1.0, 1441: 1.0}), label=1),
 Row(id=39, features=SparseVector(1952, {0: 8.0, 2: 15.0, 3: 20.0, 4: 115.0, 5: 24.0, 6: 8.0, 7: 23.0, 8: 24.0, 9: 2.0, 10: 2.0, 12: 20.0, 15: 1.0, 1508: 1.0}), label=0),
 Row(id=108, features=SparseVector(1952, {1: 5.0, 2: 30.0, 3: 4.0, 7: 5.0, 8: 4.0, 12: 5.0, 14: 1.0, 1401: 1.0}), label=0),
 Row(id=121, features=SparseVector(1952, {0: 1.0, 2: 31.0, 3: 9.0, 4: 226.0, 5: 22.0, 6: 32.0, 7: 13.0, 8: 177.0, 9: 1.0, 10: 7.0, 12: 11.0, 15: 1.0, 1441: 1.0}), label=1),
 Row(id=135, features=SparseVector(1952, {2: 17.0, 3: 3.0, 4: 19811.0, 7: 3.0, 8: 54.0, 12: 3.0, 13: 1.0, 1415: 1.0}), label=0)]

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [20]:
from pyspark.sql.window import Window

In [21]:
# window = Window().orderBy('id')
# temp_df = df.withColumn('percent_rank', F.percent_rank().over(window))
# temp_df.toPandas().tail()

In [22]:
def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    parts = []
    
    window = Window().orderBy(split_col)
    temp_df = df.withColumn('percent_rank', F.percent_rank().over(window))
    
    min_, max_ = -0.1, 0
    for i in range(len(parts_fractions)):
        max_ += parts_fractions[i]
        part = temp_df.filter((F.col('percent_rank')>min_)&(F.col('percent_rank')<=max_)).drop('percent_rank')
        parts.append(part)
        min_ = max_
    
    return parts

In [24]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [24]:
N = df.count()

In [25]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999997818413463, 0.10000010907932685, 0.10000010907932685)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [25]:
from pyspark.ml.classification import LogisticRegression


lr = LogisticRegression()
lr_model = lr.fit(train_df)

In [26]:
lr_model.transform(val_df).select('probability', 'prediction').take(5)

[Row(probability=DenseVector([0.5907, 0.4093]), prediction=0.0),
 Row(probability=DenseVector([0.8189, 0.1811]), prediction=0.0),
 Row(probability=DenseVector([0.5675, 0.4325]), prediction=0.0),
 Row(probability=DenseVector([0.84, 0.16]), prediction=0.0),
 Row(probability=DenseVector([0.8495, 0.1505]), prediction=0.0)]

In [28]:
lr_model.transform(val_df).rdd.take(5)

[Row(id=455266589666, features=SparseVector(1951, {2: 14.0, 3: 11.0, 4: 1435.0, 5: 37.0, 6: 6.0, 7: 27.0, 8: 180.0, 10: 5.0, 12: 11.0, 13: 1.0, 1419: 1.0}), label=0, rawPrediction=DenseVector([0.9938, -0.9938]), probability=DenseVector([0.7298, 0.2702]), prediction=0.0),
 Row(id=455266589668, features=SparseVector(1951, {0: 1.0, 3: 2.0, 4: 267.0, 5: 2.0, 6: 1.0, 7: 2.0, 8: 2.0, 9: 1.0, 10: 1.0, 12: 2.0, 13: 1.0, 1429: 1.0}), label=0, rawPrediction=DenseVector([1.2007, -1.2007]), probability=DenseVector([0.7687, 0.2313]), prediction=0.0),
 Row(id=455266589711, features=SparseVector(1951, {0: 6.0, 1: -1.0, 4: 1365.0, 5: 38.0, 6: 24.0, 7: 41.0, 8: 193.0, 10: 4.0, 11: 2.0, 13: 1.0, 1402: 1.0}), label=1, rawPrediction=DenseVector([1.4681, -1.4681]), probability=DenseVector([0.8128, 0.1872]), prediction=0.0),
 Row(id=455266589724, features=SparseVector(1951, {1: 3.0, 2: 16.0, 3: 37.0, 4: 5909.0, 5: 1607.0, 6: 2.0, 7: 47.0, 8: 642.0, 10: 2.0, 12: 37.0, 14: 1.0, 1411: 1.0}), label=0, rawPredic

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [30]:
lr_model.transform(val_df) \
    .rdd.map(lambda arr: (arr['probability'][1], arr['label'])) \
    .take(5)

[(0.40928328810864745, 1.0),
 (0.1811423684089621, 0.0),
 (0.43250442595258115, 0.0),
 (0.16002508793545098, 1.0),
 (0.15049681871629716, 0.0)]

In [27]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics

eps = 0.01

def predict(model, df):
    return lr_model.transform(val_df) \
                    .rdd.map(lambda arr: (float(arr['probability'][1]), float(arr['label'])))

def rocauc(model, df):
    prediction = predict(model, df)
    return BinaryClassificationMetrics(prediction).areaUnderROC

def logloss_func(arr):
    pred, label = arr
    return -(label*np.log(np.clip(pred, eps, 1.-eps)) + 
                             (1.-label)*np.log(1.-np.clip(pred, eps, 1-eps)))

def logloss(model, df):
    prediction = predict(model, df)
    return prediction.map(logloss_func).mean()

def ne(model, df):
    prediction = predict(model, df)
    label_mean = prediction.map(lambda arr: arr[1]).mean()
    
    extremum = logloss_func((label_mean, label_mean))
    return logloss(model, df)/extremum

In [32]:
logloss(lr_model, val_df)

0.5278308128014816

In [33]:
rocauc(lr_model, val_df)

0.7022402903865085

In [34]:
ne(lr_model, val_df)

0.9181830971127667

In [35]:
rocauc(lr_model, test_df)

0.7022402903865085

In [36]:
ne(lr_model, test_df)

0.9181830971127667

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [37]:
import pandas as pd

In [35]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

test_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

test_df = test_df.fillna(0, subset=num_columns)
loaded_pipeline_model = PipelineModel.load(DATA_PATH+'/pipeline_model')
test_df = loaded_pipeline_model.transform(test_df)
test_df = test_df.select('id', 'features')

In [31]:
pred = lr_model.transform(test_df).select('id','probability')
# pred = pred.toPandas()
# pred['proba'] = pred['probability'].map(lambda arr: arr[1])

In [32]:
pred.take(5)

Py4JJavaError: An error occurred while calling o812.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 155.0 failed 1 times, most recent failure: Lost task 0.0 in stage 155.0 (TID 184, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at o